In [1]:
#Tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, activations
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

#Pandas and numpy for data formats
import pandas as pd
import numpy as np


#glob for data import
import glob
import random
#PyTorch
import torch
import torch.nn as nn
#SK Learn
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
#from sklearn.utils import shuffle
import math
import sys
#MatPlotLib
import matplotlib.pyplot as plt

from models import simpleRegression
from models import feedForward, sequentialModel, sequentialDropout, lstmModel, cnnModel, cnnModelFilters, scoringSystem

#import functions
from dataImport import importTrainingData, importParamsData, importTestData, importTrainParameter, importTargetParameter, importEvalParameter
from preprocessing import preprocessData, createTrainTensors, createParamsTensors, preprocessParams, rescaleData, preprocessParameters


#importTrainData()
train_data = np.load("../train_40000.npy")
params_data = np.load("../train_target_40000.npy")
train_parameter = np.load("../train_parameter_40000.npy")
#
validation_data = np.load("../val_train_40000.npy")
validation_params = np.load("../val_target_40000.npy")
validation_parameter = np.load("../val_parameter_40000.npy")

#Create stats for zScore
mean_value_target = np.mean(params_data)
standard_deviation_target = np.std(params_data)

mean_value = np.mean(train_data)
standard_deviation = np.std(train_data)
measurements_to_remove = 30


median_train = False
replaceValuesBiggerOne_train=False
replaceFirstXValues_train=False
removeFirstXValues_train=True
normalizeData_train=False
zScore_train=True

train_data = preprocessData(train_data,
                          median=median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

train_target = preprocessData(params_data,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)

validation_data = preprocessData(validation_data,
                          median = median_train,
                          replaceValuesBiggerOne = replaceValuesBiggerOne_train,
                          replaceFirstXValues = replaceFirstXValues_train,
                          removeFirstXValues = removeFirstXValues_train,
                          normalizeData = normalizeData_train,
                          zScore = zScore_train,
                          zScore_mean = mean_value,
                          zScore_sd = standard_deviation,
                          number_to_remove = measurements_to_remove)

validation_target = preprocessData(validation_params,
                          median=False,
                          replaceValuesBiggerOne=False,
                          replaceFirstXValues=False,
                          removeFirstXValues=False,
                          normalizeData=False,
                          zScore=False,
                          zScore_mean=mean_value_target,
                          zScore_sd=standard_deviation_target,
                          number_to_remove=measurements_to_remove)


number_of_measurements = 300
if removeFirstXValues_train == True:
    number_of_measurements = 300 - measurements_to_remove    
    
#reshaping train data
if type(train_data) != np.ndarray:
    train_data = train_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:
    train_data = train_data.reshape(-1, 55, number_of_measurements)

#reshaping target data
if type(params_data) != np.ndarray:
    params_data = params_data.to_numpy()

#reshaping val data
if type(validation_data) != np.ndarray:
    validation_data = validation_data.to_numpy().reshape(-1, 55, number_of_measurements)
else:    
    validation_data = validation_data.reshape(-1, 55, number_of_measurements)

#reshaping val target
if type(validation_params) != np.ndarray:
    validation_params = validation_params.to_numpy()

    
def scaleToMaxToOne(dat, min, max):
    scaled_dat = dat / max
    return scaled_dat

def unscaleMaxToOne(scaled_dat, min, max):
    unscaled_dat = scaled_dat * max
    return unscaled_dat

min_target = np.min(train_target) 
max_target = np.max(train_target)
print(f"Rescaling targets with max = {max_target}")
train_target = scaleToMaxToOne(train_target, min_target, max_target)
val_target = scaleToMaxToOne(validation_target, min_target, max_target)    

prevent_overfitting = keras.callbacks.EarlyStopping(monitor="val_loss", patience = 20, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, min_delta=0.0001)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Rescaling targets with max = 0.6713607926990712


In [2]:
def sequentialModel(av = 'selu'):
    model = keras.Sequential([
        keras.layers.InputLayer((55, number_of_measurements)),
        keras.layers.Flatten(),
        layers.Dense(1024),
        layers.Activation(av),
        layers.Dense(1024),
        layers.Activation(av),
        layers.Dense(1024),
        layers.Activation(av),
        layers.Dense(1024),
        layers.Activation(av),
        layers.Dense(1024),
        layers.Activation(av),
        layers.Dense(1024),
        layers.Activation(av),
        layers.Dense(55)
    ])

    optim = keras.optimizers.Adam(learning_rate=0.0001)
    
    model.compile(optimizer=optim,
                  loss='mae')

    return model

model = sequentialModel()
model2 = sequentialModel()
model3 = sequentialModel()

# Model 1

In [3]:
history_test_1 = model.fit(train_data,
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    #validation_data= (validation_data, validation_params),
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 6s 5ms/step - loss: 0.2206 - val_loss: 0.0939
Epoch 2/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0893 - val_loss: 0.0819
Epoch 3/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0733 - val_loss: 0.0661
Epoch 4/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0645 - val_loss: 0.0634
Epoch 5/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0590 - val_loss: 0.0594
Epoch 6/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0536 - val_loss: 0.0626
Epoch 7/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0507 - val_loss: 0.0543
Epoch 8/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0468 - val_loss: 0.0455
Epoch 9/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0452 - val_loss: 0.0507
Epoch 10/1000
1000/1000 [==============================] - 5s 5m

1000/1000 [==============================] - 5s 5ms/step - loss: 9.1477e-04 - val_loss: 0.0071
Epoch 79/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 9.1809e-04 - val_loss: 0.0071
Epoch 80/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 9.1379e-04 - val_loss: 0.0071
Epoch 81/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 9.1753e-04 - val_loss: 0.0070
Epoch 82/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 9.2163e-04 - val_loss: 0.0070
Epoch 83/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 9.1434e-04 - val_loss: 0.0070
Epoch 84/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 9.1530e-04 - val_loss: 0.0071
Epoch 85/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 9.1593e-04 - val_loss: 0.0070
Epoch 86/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 9.1065e-04 - val_loss: 0.0071
Epoch 87/1000
1000/1000 [========

In [4]:
pred = model.predict(validation_data)
#pred = pred * standard_deviation_target + mean_value_target 


pred_final = unscaleMaxToOne(pred, min_target, max_target)
score = scoringSystem(validation_params, pred_final)
print(score)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9684.322856770996, shape=(), dtype=float64)


In [5]:
last_five_idx = len(history_test_1.history["val_loss"])-5
val_loss_1 = sum(history_test_1.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_1.history["loss"])-5
train_loss_1 = sum(history_test_1.history["loss"][last_five_idx:])/5

# Model 2

In [6]:
history_test_2 = model2.fit(train_data,
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    #validation_data= (validation_data, validation_params),
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 6s 5ms/step - loss: 0.2170 - val_loss: 0.0921
Epoch 2/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0898 - val_loss: 0.0822
Epoch 3/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0763 - val_loss: 0.0711
Epoch 4/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0677 - val_loss: 0.0558
Epoch 5/1000
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0615 - val_loss: 0.0615
Epoch 6/1000
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0553 - val_loss: 0.0489
Epoch 7/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0515 - val_loss: 0.0545
Epoch 8/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0486 - val_loss: 0.0508
Epoch 9/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0471 - val_loss: 0.0451
Epoch 10/1000
1000/1000 [==============================] - 5s 5m

In [7]:
pred2 = model2.predict(validation_data)
#pred = pred * standard_deviation_target + mean_value_target 


pred_final2 = unscaleMaxToOne(pred2, min_target, max_target)
score2 = scoringSystem(validation_params, pred_final2)
print(score2)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9618.17766500301, shape=(), dtype=float64)


In [8]:
last_five_idx = len(history_test_2.history["val_loss"])-5
val_loss_2 = sum(history_test_2.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_2.history["loss"])-5
train_loss_2 = sum(history_test_2.history["loss"][last_five_idx:])/5

# Model 3

In [9]:
history_test_3 = model3.fit(train_data,
                    train_target,
                    batch_size=32,
                    epochs = 1000,
                    #validation_data= (validation_data, validation_params),
                    validation_split = 0.2,
                    shuffle = True,
                    callbacks=[prevent_overfitting, reduce_lr])

Epoch 1/1000
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2189 - val_loss: 0.1103
Epoch 2/1000
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0932 - val_loss: 0.0820
Epoch 3/1000
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0777 - val_loss: 0.0799
Epoch 4/1000
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0701 - val_loss: 0.0793
Epoch 5/1000
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0628 - val_loss: 0.0672
Epoch 6/1000
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0583 - val_loss: 0.0534
Epoch 7/1000
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0539 - val_loss: 0.0573
Epoch 8/1000
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0516 - val_loss: 0.0425
Epoch 9/1000
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0473 - val_loss: 0.0441
Epoch 10/1000
1000/1000 [==============================] - 6s 6m

In [10]:
pred3 = model3.predict(validation_data)
#pred = pred * standard_deviation_target + mean_value_target 


pred_final3 = unscaleMaxToOne(pred3, min_target, max_target)
score3 = scoringSystem(validation_params, pred_final3)
print(score3)

tf.Tensor(550000.0, shape=(), dtype=float64)
tf.Tensor(9679.452789076098, shape=(), dtype=float64)


In [11]:
last_five_idx = len(history_test_3.history["val_loss"])-5
val_loss_3 = sum(history_test_3.history["val_loss"][last_five_idx:])/5

last_five_idx = len(history_test_3.history["loss"])-5
train_loss_3 = sum(history_test_3.history["loss"][last_five_idx:])/5

# Evaluation

In [12]:
val_loss_final = (val_loss_1 + val_loss_2 + val_loss_3)/3
train_loss_final = (train_loss_1 + train_loss_2 + train_loss_3)/3
score_final = (score + score2 + score3)/3

In [13]:
print("Final train loss: ", train_loss_final)
print("Final validation loss: ", val_loss_final)
print("Final Score: ", score_final)

Final train loss:  0.0011815491132438182
Final validation loss:  0.0076211907900869845
Final Score:  tf.Tensor(9660.651103616701, shape=(), dtype=float64)
